<h1>Block algorthims using dask array</h1>

Blocked algorithms allow you to process data too big to fit into memory.  Dask uses these to both tackle huge datasets and to parallelize the work.<br><br>
Blocked mean example

In [ ]:
x = h5py.File('myfile.hdf5')['x']             #Trillion element array on disk
sums = []                                     
counts = []
for i in range(1_000_000):                    #Loop through array 1 million times
    chunk = x[1_000_000*i: 1_000_000*(i+1)]   #Pull out each chunk
    sums.append(np.sum(chunk))                #Sum chunk
    counts.append(len(chunk))                 #Count chunk
    
result=sum(sums)/sum(counts)                  #Aggregate results

<br><br>Dask implements a subset of ndarray's api using blocked algorithms to distribute the data/processing 
across multiple ndarrays

<br>Block algorthims like above can be split into parallel operations because there are no dependencies.  This allows you to process data that fits on disk, but maybe not in memory.


<h3>Example Numpy ndarray operations</h3>

Create a numpy ndarray and fill with random numbers to simulate data

In [ ]:
import numpy as np
x = np.random.random((10000, 10000))
x


In [ ]:
print("Size in GB:",x.nbytes / 1e9) 

In [ ]:
%%time
#Perform some operations:
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

<br><h3>Now let's do it in dask array</h3>

In [ ]:
#Import dask array and create same random data, breaking into chunks.
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

In [ ]:
%%time
#Same operation, but lazy compute, so returns instantly
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

In [ ]:
z.visualize()

In [ ]:
%%time
z.compute()

<br><h4>Let's try again, but with a 10 trillion elements</h4>

By default dask uses a local threaded cluster which parallelizes operations and limits memory usage but is limited to a single core, so processor intensive applications won't see much performance improvement (IO bound ones will).<br>You can easily change this to a multi-process local cluster by importing the distributed package which scales from local machine, to ad hoc clusters, to cloud services to HPC.

In [ ]:
#Import distributed and create a local 4 process client with restricted memory usage.  
from dask.distributed import Client, progress
client = Client(processes=True, threads_per_worker=4,
                n_workers=2, memory_limit='1GB')

In [ ]:
client

In [ ]:
import dask.array as da
x = da.random.random((100000, 100000), chunks=(1000, 1000))#10 trillion
x

In [ ]:
#Same operation, lazy compute
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

In [ ]:
z.compute()

<h4>Dask arrays allow for reduced memory footprint and parallel processing of blocked algorithms</h4>

In [ ]:

#Tab completion shows api implemented
import dask.array as da
da.

Close down the cluster

In [ ]:
client.shutdown()